In [1]:
# Импортируем необходимые библиотеки

import telegram # библиотека для работы с Telegram
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import io # библиотека для управления потоками ввода/вывода данных (чтобы  пересылать графики из буфера обмена и не сохранять их лишний раз)
import logging
import pandas as pd
import pandahouse
# from read_db.CH import Getch # импортируем коннектор, чтобы подключиться к базе из Jupyter'а. (НЕ РАБОТАЕТ! говорит, нет read_db)
import os

In [3]:
# Создаем бота с помощью нашего токена
bot = telegram.Bot(token='5227151149:AAGkhDIQK5PZo1i88XKZ22INjOnBXq_06Z4')

Чтобы бот смог отправлять нам сообщения, нужно самим написать (или просто запустить) что-то боту из своего аккаунта в Телеграме. Сделали это.

In [4]:
# Посмотрим последние обновления в боте, чтобы увидеть id, куда бот будет отправлять сообщения
updates = bot.getUpdates()

print(updates[-1])

{'my_chat_member': {'date': 1652003913, 'old_chat_member': {'user': {'username': 'evg_kc_report_bot', 'first_name': 'KC report bot (evg.)', 'id': 5227151149, 'is_bot': True}, 'status': 'left', 'until_date': None}, 'new_chat_member': {'user': {'username': 'evg_kc_report_bot', 'first_name': 'KC report bot (evg.)', 'id': 5227151149, 'is_bot': True}, 'status': 'member', 'until_date': None}, 'chat': {'all_members_are_administrators': False, 'title': '20.04.22 karpov.courses Симулятор аналитика | АЛЕРТЫ', 'type': 'group', 'id': -691600284}, 'from': {'username': 'evg_dbrvn', 'first_name': 'Evg.', 'language_code': 'ru', 'id': 244991955, 'is_bot': False}}, 'update_id': 19389167}


In [3]:
# Сохраним id, куда бот будет отправлять отчеты (наш id)
chat_id = 244991955
# id чата '20.04.22 karpov.courses Симулятор аналитика | ОТЧЕТЫ'
# chat_id = -799682816

In [4]:
# Отправим сообщение от бота по указанному id
msg = 'Hi!'

bot.sendMessage(chat_id=chat_id, text=msg)

### Отправим график с помощью бота

In [ ]:
# Зададим параметры тестового графика
x = np.arange(1, 10, 1)
y = np.random.choice(10, len(x))

In [ ]:
# Пересылаем график с помощью пакета io
sns.lineplot(x, y)
plt.title('test plot')

plot_object = io.BytesIO() # создаем файловый объект - картинку
plt.savefig(plot_object) # сохраняем в него наш график
plot_object.name = 'test_plot.png' # задаем имя нашего файлового объекта
plot_object.seek(0) # переносим курсор из конца файлового объекта в начало, чтобы прочитать весь файл (чтобы избежать отправки боту пустого файла)

plt.close()

# Отправляем изображение
bot.sendPhoto(chat_id=chat_id, photo=plot_object)

### Отправим файл с помощью бота

In [5]:
# Подключаемся к БД
connection = {
    'host': 'https://clickhouse.lab.karpov.courses',
    'password': 'dpo_python_2020',
    'user': 'student',
    'database': 'simulator_20220420'
}

In [9]:
# Запрос
# Все данные за вчера
q = '''SELECT *
       FROM {db}.feed_actions 
       WHERE toDate(time) = yesterday()
       ORDER BY time DESC
       LIMIT 100'''

In [10]:
# Сохраним результат запроса в pandas df
test_df = pandahouse.read_clickhouse(q, connection=connection)
test_df

,user_id,post_id,action,time,gender,age,country,city,os,source,exp_group
0,143977,3598,view,2022-04-25 23:59:59,1,22,Russia,Mytishchi,Android,organic,1
1,154475,3567,view,2022-04-25 23:59:59,0,27,Russia,Alkhan-Kala,Android,organic,2
2,151363,3645,view,2022-04-25 23:59:58,0,29,Russia,Voronezh,Android,organic,3
3,145964,3628,view,2022-04-25 23:59:58,1,17,Russia,Kamyshin,Android,organic,1
4,145964,3591,view,2022-04-25 23:59:58,1,17,Russia,Kamyshin,Android,organic,1
...,...,...,...,...,...,...,...,...,...,...,...
95,154711,3585,view,2022-04-25 23:59:45,1,43,Russia,Petropavlovka,Android,organic,2
96,129004,3625,like,2022-04-25 23:59:45,1,28,Russia,Elista,iOS,organic,3
97,8164,3547,view,2022-04-25 23:59:45,0,22,Russia,Moscow,Android,ads,3
98,144161,3462,like,2022-04-25 23:59:44,1,40,Russia,Shchigry,iOS,organic,0


In [11]:
file_object = io.StringIO() # создаем файловый объект - текст
test_df.to_csv(file_object) # переводим в csv-формат
file_object.seek(0) # переносим курсор из конца файлового объекта в начало
file_object.name = 'test_doc.csv' # задаем имя нашего файлового объекта

# Отправляем файл
bot.sendDocument(chat_id=chat_id, document=file_object)

Fin.